<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/WaterSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import itertools
import copy
import numpy as np
import heapq

In [ ]:
class State:
  def __init__(self, bottles, parent = None, capacity = 0):
    self.bottles = bottles
    self.parent = parent
    if parent is None:
      self.capacity = max(len(bottle) for bottle in self.bottles)
      self.distance = 0
    else:
      self.capacity = parent.capacity
      self.distance = parent.distance + 1

  # Check if this is the goal
  def is_goal(self):
    return all(len(bottle) == 0 or len(bottle) == self.capacity and len(set(bottle)) == 1 for bottle in self.bottles)

  # Find all pourable bottle pairs
  def pourable_pairs(self):
    result = []
    for i in range(len(self.bottles)-1,-1,-1):
      for j in range(len(self.bottles)-1,-1,-1):
        if i != j:
          # One is containing and one is empty
          if len(self.bottles[i]) > 0 and len(self.bottles[j]) == 0: 
            result.append((i,j))
          # Two have the same tops
          if len(self.bottles[i]) > 0 and len(self.bottles[j]) > 0 and self.bottles[i][-1] == self.bottles[j][-1] and len(self.bottles[j]) < self.capacity:
            result.append((i,j))
    return result

  # Expand a state
  def expand(self):
    # If this state is expandible
    children = []
    for pair in self.pourable_pairs():
      
      child = State(copy.deepcopy(self.bottles), self)
      top = child.bottles[pair[0]][-1]
      while len(child.bottles[pair[0]]) > 0 and child.bottles[pair[0]][-1] == top and len(child.bottles[pair[1]]) < self.capacity:
        child.bottles[pair[0]].pop()
        child.bottles[pair[1]].append(top)
      children.append(child)
    return children

  # Estimated f = g + h
  def cost(self):
    h = 0
    for bottle in self.bottles:
      for i in range (len(bottle)-1):
        if bottle[i] != bottle[i+1]: h += 1
    return h + self.distance
  
  # Overide the comparators
  def __lt__(self, other):
    return self.cost() < other.cost()

  def __eq__(self, other):
    return self.bottles == other.bottles
   
  # Get path from root
  def getpath(self):
    path = [self]
    while path[-1].parent != None: path.append(path[-1].parent)
    return path

In [ ]:
level1 = State([['O'], ['O', 'O', 'O']])
level2 = State([['B','O','B','O'],['O','B','O','B'],[]])
level3 = State([['O', 'O', 'R', 'B'], ['B', 'O', 'R', 'B'], ['R','B','O','R'],[],[]])
level4 = State([['O', 'B', 'R', 'O'], ['B', 'R', 'O', 'O'], ['B','R','B','R'],[],[]])
level5 = State([['P','B','G','O'],['R','O','R','P'],['B','R','G','G'],['P','B','O','G'],['B','R','P','O'],[],[]])
level6 = State([['B','P','G','G'],['B','O','R','G'],['O','B','P','B'],['R','O','R','P'],['O','G','R','P'],[],[]])
level7 = State([['R','R','O','B'],['P','B','P','O'],['P','B','R','P'],['O','G','R','G'],['B','G','G','O'],[],[]])
level8 = State([['O','P','G','P'],['O','R','R','B'],['B','B','G','R'],['O','G','R','O'],['G','O','P','B'],[],[]])
level9 = State([['G','O','B','R'],['O','P','R','G'],['O','B','R','G'],['P','B','B','G'],['P','P','R','O'],[],[]])
level10 = State([['B','B','O','P'],['O','P','Y','P'],['R','G','Y','GR'],['GR','R','GR','G'],['B','B','Y','G'],['GR','O','R','Y'],['G','R','O','P'],[],[]])
level11 = State([['Y','P','GR','Y'],['G','R','B','Y'],['P','GR','O','R'],['O','G','G','Y'],['B','P','G','R'],['GR','R','B','O'],['P','O','B','GR'],[],[]])
level12 = State([['R','G','G','G'],['O','R','P','G'],['P','O','R','O'],['B','P','O','P'],['B','B','B','O']])
level13 = State([['R','O','G','P'],['GR','O','GR','G'],['B','GR','P','O'],['P','G','Y','R'],['GR','B','B','B'],['R','Y','P','R'],['O','Y','G','Y'],[],[]])
level14 = State([['Y','GR','B','Y'],['P','B','R','G'],['B','P','G','B'],['Y','R','GR','O'],['G','O','Y','G'],['R','P','O','R'],['O','P','GR','GR'],[],[]])
level15 = State([['G','O','R','O'],['B','B','R','O'],['P','P','B','O'],['G','P','R','B'],['G','R','G','P'],[],[]])
level16 = State([['Y','Y','B','G'],['GR','R','O','P'],['R','G','O','B'],['GR','B','R','P'],['GR','O','P','P'],['G','Y','B','Y'],['G','GR','R','O'],[],[]])
level17 = State([['B','B','GR','P'],['Y','R','GR','P'],['R','R','GR','B'],['G','Y','Y','O'],['G','O','GR','O'],['O','P','GR','R'],['B','Y','P','G'],[],[]])
level18 = State([['P','R','B','P'],['R','G','G','O'],['R','O','O','P'],['P','R','G','B'],['G','O','B','B'],[],[]])
level19 = State([['R','P','P','R'],['Y','R','B','R'],['O','G','O','O'],['GR','B','G','GR'],['G','Y','GR','O'],['Y','GR','P','Y'],['P','B','B','G'],[],[]])
level20 = State([['P','B','G','B'],['O','GR','P','R'],['B','Y','Y','G'],['P','O','O','G'],['GR','GR','G','R'],['B','R','Y','Y'],['R','P','O','GR'],[],[]])

In [ ]:
class WaterSort:
  def __init__ (self, initial_state):
    self.initial_state = initial_state

  def DFShelper(self, stack, visited, loop, max_stack_size):
    while len(stack) > 0:
      
      max_stack_size = max(len(stack), max_stack_size)
      current = stack.pop()

      # Just to check if initial state is also goal
      if current.is_goal(): return current, loop, max_stack_size

      visited.append(current)
      children = current.expand()
      for child in children:
        if child.is_goal(): return child, loop, max_stack_size
        if child not in visited: stack.append(child)
      
      loop += 1

    return None, loop, max_stack_size

  def Astarhelper(self, heap, visited, loop, max_heap_size):
    while len(heap) > 0:
      max_heap_size = max(max_heap_size, len(heap))
      current = heapq.heappop(heap)

      # Just to check if initial state is also goal
      if current.is_goal(): return current, loop, max_heap_size

      visited.append(current)
      children = current.expand()
      # for child in children: print(child.bottles)

      for child in children:
        if child.is_goal(): return child, loop, max_heap_size
        if child not in visited: heap.append(child)
        heapq.heapify(heap) 
      
      loop += 1

    return None, loop, max_heap_size

  def DFS(self):
    goal, loop, max_stack_size = self.DFShelper([self.initial_state], [], 0, 0)
    if goal is None: print("No solution")
    else:
      path = [goal]
      while path[-1].parent is not None: path.append(path[-1].parent)
      path.reverse()
      print("DFS executed after", loop, "loops", "with max stack size", max_stack_size)
      for i in range(len(path)): print("Step", i, ":", path[i].bottles)

  def Astar(self):
    goal, loop, max_heap_size = self.Astarhelper([self.initial_state], [], 0, 0)
    if goal is None: print("No solution")
    else:
      path = [goal]
      while path[-1].parent is not None: path.append(path[-1].parent)
      path.reverse()
      print("Astar executed after", loop, "loops", "with max stack size", max_heap_size)
      for i in range(len(path)): print("Step", i, ":", path[i].bottles)


In [ ]:
ws = WaterSort(level3)
ws.Astar()

Astar executed after 341 loops with max stack size 650
Step 0 : [['O', 'O', 'R', 'B'], ['B', 'O', 'R', 'B'], ['R', 'B', 'O', 'R'], [], []]
Step 1 : [['O', 'O', 'R', 'B'], ['B', 'O', 'R', 'B'], ['R', 'B', 'O'], ['R'], []]
Step 2 : [['O', 'O', 'R', 'B'], ['B', 'O', 'R', 'B'], ['R', 'B'], ['R'], ['O']]
Step 3 : [['O', 'O', 'R'], ['B', 'O', 'R', 'B'], ['R', 'B', 'B'], ['R'], ['O']]
Step 4 : [['O', 'O', 'R'], ['B', 'O', 'R'], ['R', 'B', 'B', 'B'], ['R'], ['O']]
Step 5 : [['O', 'O', 'R'], ['B', 'O'], ['R', 'B', 'B', 'B'], ['R', 'R'], ['O']]
Step 6 : [['O', 'O'], ['B', 'O'], ['R', 'B', 'B', 'B'], ['R', 'R', 'R'], ['O']]
Step 7 : [['O', 'O', 'O'], ['B'], ['R', 'B', 'B', 'B'], ['R', 'R', 'R'], ['O']]
Step 8 : [['O', 'O', 'O'], ['B', 'B', 'B', 'B'], ['R'], ['R', 'R', 'R'], ['O']]
Step 9 : [['O', 'O', 'O'], ['B', 'B', 'B', 'B'], ['R', 'R', 'R', 'R'], [], ['O']]
Step 10 : [['O', 'O', 'O', 'O'], ['B', 'B', 'B', 'B'], ['R', 'R', 'R', 'R'], [], []]


In [ ]:
ws.DFS()

DFS executed after 18 loops with max stack size 64
Step 0 : [['O', 'O', 'R', 'B'], ['B', 'O', 'R', 'B'], ['R', 'B', 'O', 'R'], [], []]
Step 1 : [['O', 'O', 'R'], ['B', 'O', 'R', 'B'], ['R', 'B', 'O', 'R'], ['B'], []]
Step 2 : [['O', 'O'], ['B', 'O', 'R', 'B'], ['R', 'B', 'O', 'R'], ['B'], ['R']]
Step 3 : [['O', 'O'], ['B', 'O', 'R'], ['R', 'B', 'O', 'R'], ['B', 'B'], ['R']]
Step 4 : [['O', 'O'], ['B', 'O'], ['R', 'B', 'O', 'R'], ['B', 'B'], ['R', 'R']]
Step 5 : [[], ['B', 'O', 'O', 'O'], ['R', 'B', 'O', 'R'], ['B', 'B'], ['R', 'R']]
Step 6 : [['O', 'O', 'O'], ['B'], ['R', 'B', 'O', 'R'], ['B', 'B'], ['R', 'R']]
Step 7 : [['O', 'O', 'O'], [], ['R', 'B', 'O', 'R'], ['B', 'B', 'B'], ['R', 'R']]
Step 8 : [[], ['O', 'O', 'O'], ['R', 'B', 'O', 'R'], ['B', 'B', 'B'], ['R', 'R']]
Step 9 : [['R'], ['O', 'O', 'O'], ['R', 'B', 'O'], ['B', 'B', 'B'], ['R', 'R']]
Step 10 : [[], ['O', 'O', 'O'], ['R', 'B', 'O'], ['B', 'B', 'B'], ['R', 'R', 'R']]
Step 11 : [['O', 'O', 'O'], [], ['R', 'B', 'O'], ['B',

In [ ]:
state = State(level5, None)
path = DFS([state], 0, [])

for i in range(len(path)):
  print("Step", i, ":", path[i].bottles)

DFS executed after 93 loops
Step 0 : [['P', 'B', 'G', 'O'], ['R', 'O', 'R', 'P'], ['B', 'R', 'G', 'G'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], [], []]
Step 1 : [['P', 'B', 'G'], ['R', 'O', 'R', 'P'], ['B', 'R', 'G', 'G'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], ['O'], []]
Step 2 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R', 'G', 'G'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], ['O'], ['G']]
Step 3 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B', 'O', 'G'], ['B', 'R', 'P', 'O'], ['O'], ['G', 'G', 'G']]
Step 4 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B', 'O'], ['B', 'R', 'P', 'O'], ['O'], ['G', 'G', 'G', 'G']]
Step 5 : [['P', 'B'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B'], ['B', 'R', 'P', 'O'], ['O', 'O'], ['G', 'G', 'G', 'G']]
Step 6 : [['P'], ['R', 'O', 'R', 'P'], ['B', 'R'], ['P', 'B', 'B'], ['B', 'R', 'P', 'O'], ['O', 'O'], ['G', 'G', 'G', 'G']]
Step 7 : [['P', 'P'], ['R', 'O', 'R'], ['B', 'R'], ['P', 'B', 'B'], ['B', 'R', 'P', 'O'], ['O', 'O

In [ ]:
input1 = State([['X','C','X','C'],['C','X','C','X'],[]], None)

path = Astar([input1], 0, [])

for i in range(len(path)):
  print("Step", i, ":", path[i].bottles)


A* executed after 13 loops
Step 0 : [['X', 'C', 'X', 'C'], ['C', 'X', 'C', 'X'], []]
Step 1 : [['X', 'C', 'X', 'C'], ['C', 'X', 'C'], ['X']]
Step 2 : [['X', 'C', 'X'], ['C', 'X', 'C', 'C'], ['X']]
Step 3 : [['X', 'C'], ['C', 'X', 'C', 'C'], ['X', 'X']]
Step 4 : [['X', 'C', 'C', 'C'], ['C', 'X'], ['X', 'X']]
Step 5 : [['X', 'C', 'C', 'C'], ['C'], ['X', 'X', 'X']]
Step 6 : [['X'], ['C', 'C', 'C', 'C'], ['X', 'X', 'X']]
Step 7 : [['X', 'X', 'X', 'X'], ['C', 'C', 'C', 'C'], []]
